In [1]:
import pandas as pd

In [10]:
# Importamos los archivos
pd_inven67 = pd.read_csv('INVENTOR6724.csv')
pd_MX = pd.read_excel('INVENTORES_MX_USPTO_version para stata (1).xlsx')

In [11]:
# Creamos una columna con el nombre completo del inventor
pd_inven67["NOMBRE_INVENTOR_1"] = (
    pd_inven67["inventor_last_name"].str.strip() + "," + 
    pd_inven67["inventor_first_name"].str.strip()
)

# Creamos una columna con el nombre completo del inventor
pd_MX["NOMBRE_INVENTOR_2"] = (
    pd_MX["Inventor"]
)

In [12]:
# Creamos un DataFrame con los nombres de los inventores
df_Nombres = pd.DataFrame({
    "NOMBRE_INVENTOR_1": pd_inven67["NOMBRE_INVENTOR_1"],
    "NOMBRE_INVENTOR_2": pd_MX["NOMBRE_INVENTOR_2"],
})

In [13]:
# Limpiamos los nombres del segundo DataFrame
df_Nombres["NOMBRE_INVENTOR_2"] = df_Nombres["NOMBRE_INVENTOR_2"].fillna('').apply(lambda x: ','.join([' '.join(x.split()[-2:]), ' '.join(x.split()[:-2])]))

In [ ]:
# Cargamos los casos especiales
df_casoEspecial = pd.read_csv('casos_con_mas_de_tres_palabras.csv')

# Hacemos un merge para actualizar los nombres incorrectos
df_Nombres = df_Nombres.merge(df_casoEspecial[['NOMBRE_INVENTOR_1', 'NOMBRE_INVENTOR_2']], on='NOMBRE_INVENTOR_1', how='left', suffixes=('', '_correcto'))

# Actualizamos los nombres incorrectos
df_Nombres['NOMBRE_INVENTOR_2'] = df_Nombres['NOMBRE_INVENTOR_2_correcto'].combine_first(df_Nombres['NOMBRE_INVENTOR_2'])

# Eliminamos la columna temporal
df_Nombres.drop(columns=['NOMBRE_INVENTOR_2_correcto'], inplace=True)

In [16]:
# Verificamos que ningún nombre de la primera columna existe en la segunda columna
df_Nombres['coincidencia'] = df_Nombres.apply(
    lambda row: row['NOMBRE_INVENTOR_1'] not in df_Nombres['NOMBRE_INVENTOR_2'].values, axis=1
)

In [ ]:
# En un dataframe llamado Listas_Completa se colocan todos los nombres 
# del 'NOMBRE_INVENTOR_2' mas los que no se encuentra en el 'NOMBRE_INVENTOR_1'
Listas_Completa = pd.concat([df_Nombres['NOMBRE_INVENTOR_2'], df_Nombres[~df_Nombres['coincidencia']]['NOMBRE_INVENTOR_1']])
Listas_Completa = Listas_Completa.drop_duplicates()
# Se le pone un encabezado a Listas_Completa llamado Nombre_Inventor
Listas_Completa = pd.DataFrame(Listas_Completa, columns=['Nombre Inventor'])

# Eliminamos duplicados
Listas_Completa = Listas_Completa.drop_duplicates()

# Guardamos archivo
# Listas_Completa.to_csv('coincidencias.csv', index=False)

In [26]:
import unicodedata

pd_NombresLimpios = df_Nombres.copy()

# Eliminamos la columna 'NOMBRE_INVENTOR_1' y 'coincidencia'
pd_NombresLimpios.drop(columns=['NOMBRE_INVENTOR_1', 'coincidencia'], inplace=True) 

# Eliminamos los datos que solamente contengan ','
pd_NombresLimpios = pd_NombresLimpios[pd_NombresLimpios['NOMBRE_INVENTOR_2'] != ',']

# Eliminamos los datos DESPUES del nombre 'N. Franklin,David' que es la fila 1318
pd_NombresLimpios = pd_NombresLimpios.iloc[:1317]


def remove_accents_and_hyphens(input_str):
    # Elimina guiones
    no_hyphens = input_str.replace('-', ' ').replace('.', '')
    # Normalización para eliminar acentos
    nfkd_str = unicodedata.normalize('NFKD', no_hyphens)
    return "".join([c for c in nfkd_str if not unicodedata.combining(c)])

pd_NombresLimpios['NOMBRE_INVENTOR_2'] = pd_NombresLimpios['NOMBRE_INVENTOR_2'].apply(
    lambda x: ','.join([
        remove_accents_and_hyphens(x.rstrip(',').split()[-1]), 
        remove_accents_and_hyphens(' '.join(x.rstrip(',').split()[:-1]))
    ])
    if x.endswith(',') else remove_accents_and_hyphens(x)
)

In [9]:
pd_coincidencias = pd.read_csv('coincidencias.csv')

# en el archivo quedaron casos como el siguiente, donde por ejemplo los nombres quedaron de la siguiente manera 'Carlos Tostado,'
# por lo que hay que poner todos los nombres que tengan esa estructura, de la siguiente forma 'Tostado,Carlos' en la columna 'Nombre Inventor'
import unicodedata

def remove_accents_and_hyphens(input_str):
    # Elimina guiones
    no_hyphens = input_str.replace('-', ' ').replace('.', '')
    # Normalización para eliminar acentos
    nfkd_str = unicodedata.normalize('NFKD', no_hyphens)
    return "".join([c for c in nfkd_str if not unicodedata.combining(c)])

pd_coincidencias['Nombre Inventor'] = pd_coincidencias['Nombre Inventor'].apply(
    lambda x: ','.join([
        remove_accents_and_hyphens(x.rstrip(',').split()[-1]), 
        remove_accents_and_hyphens(' '.join(x.rstrip(',').split()[:-1]))
    ])
    if x.endswith(',') else remove_accents_and_hyphens(x)
)

# Guardamos archivo
pd_coincidencias.to_csv('coincidencias.csv', index=False)

In [ ]:

# Ahora la columna completa 'NOMBRE_INVENTOR_2' del DataFrame df_Nombres, la copiamos y pegamos al pd_MX alado de la columna 'Inventor'

pos = pd_MX.columns.get_loc('Inventor')

# Insertamos la columna 'NOMBRE_INVENTOR_2' justo al lado de 'Inventor'
pd_MX.insert(pos + 1, 'NOMBRE_INVENTOR', pd_NombresLimpios['NOMBRE_INVENTOR_2'])

# Eliminamos la columna NOMBRE_INVENTOR_2 del DataFrame pd_MX
pd_MX.drop(columns=['NOMBRE_INVENTOR_2'], inplace=True)

In [ ]:
# Guardamos el archivo
pd_MX.to_excel('INVENTORES_MX1.xlsx', index=False)

In [38]:
pd_FaltantesID = pd.read_csv('FaltantesDeID (2).csv')
pd_Lista = pd.read_csv('coincidencias.csv')

In [36]:
# Eliminamos guiones y Normalización para eliminar acentos
pd_FaltantesID['Inventor'] = pd_FaltantesID['Inventor'].apply(
    lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x) if not unicodedata.combining(c)])
)

In [39]:
import re

def normalize_name(name):
    # Quitar comas, puntos y otros signos de puntuación
    name_clean = re.sub(r'[^\w\s]', ' ', name)
    # Convertir a minúsculas
    name_clean = name_clean.lower()
    # Separar en tokens, eliminar espacios extra y ordenar
    tokens = sorted(name_clean.split())
    # Juntar tokens en una cadena canónica
    return " ".join(tokens)

# Normalizamos los nombres en ambos DataFrames
pd_FaltantesID['normalized'] = pd_FaltantesID['Inventor'].apply(normalize_name)
pd_Lista['normalized'] = pd_Lista['Nombre Inventor'].apply(normalize_name)

# Ahora verificamos si no hay coincidencia en el DataFrame pd_Lista
pd_FaltantesID['coincidencia'] = pd_FaltantesID['normalized'].apply(
    lambda inv: inv not in pd_Lista['normalized'].values
)

In [40]:
# guardamos pd_FaltantesID
pd_FaltantesID.to_excel('FaltantesDeID EXISTENTES.xlsx', index=False)